### Challenges This Week

* With `Surprise` library, processing time too long on the large dataset
* Tried `Pyspark`, to take advantage of it's distributed processing:
    - mapping `str` type userIDs to `int` type userIDs was necessary
    - processing time for training/evaluating extremely long due to ALS algo --> Tried using a smaller subset of data, still too long
    - Terrible RMSE after evaluating baseline model (RMSE: 4.719, peviously with `Surprise`, it was 0.99-1.1)
    - pyspark session abruptly stopping due to running out of memory (?) during cross-val, even with smaller subset of data

### Next Steps

* Revert back to `Surprise` library for modeling since no need to map `str` type userIDs to `int` type, and more options for algorithms
* Most likely will have to only use a fraction of the overall dataset instead of entire dataset due to processing time


**Note**: Dataset contained 51M+ rows, and 46M+ rows after dropping rows containing `None` type values.



<hr>

Import libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, FloatType, StringType, LongType, StructField
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

Create SparkSession

In [2]:
spark = SparkSession.builder.master("local[6]").appName('Book Ratings')\
                            .config('spark.executor.memory', '8g')\
                            .config('spark.driver.memory', '4g')\
                            .getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 16:18:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Import Dataset into Spark DF

In [3]:
# Get underlying Spark Context
sc = spark.sparkContext

In [4]:
# define new schema
schema = StructType()\
        .add('bookID', IntegerType(), nullable=False)\
        .add('userID', StringType(), nullable=False)\
        .add('rating', FloatType(), nullable=False)\
        .add('timestamp', LongType(), nullable=False)

In [5]:
# Import data into PySpark DF
books = spark.read.format('csv').schema(schema).load('../data/Books.csv')
books.head()

Row(bookID=1713353, userID='A1C6M8LCIX4M6M', rating=5.0, timestamp=1123804800)

In [6]:
# Verify data types
books.printSchema()

root
 |-- bookID: integer (nullable = true)
 |-- userID: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



The data is returned in the format `(int(bookID), str(userID), float(rating), long(timestamp))`.

To parse into a `PySpark` `Rating` object, it is expected to be in the format `(int(userID), int(bookID), float(rating), long(timestamp))`.

<hr>

### Check for `NoneType` Values

In [7]:
# Check for None values in the DF
none_count = books.filter(
    (col("bookID").isNull()) |
    (col("userID").isNull()) |
    (col("rating").isNull()) |
    (col("timestamp").isNull())
).count()
has_none_values = none_count > 0

# Print the result
if has_none_values:
    print(f"The DF has at least {none_count} rows with a None value.")
else:
    print("The DF does not have any rows with None values.")

The DF has at least 6671993 rows with a None value.


In [8]:
# Verify what rows with None values look like
none_df = books.filter(
    (col('bookID').isNull()) |
    (col('userID').isNull()) |
    (col('rating').isNull()) |
    (col('timestamp').isNull())
)
none_df.take(10)

[Row(bookID=None, userID='ASS457AQPDIFZ', rating=5.0, timestamp=1409443200),
 Row(bookID=None, userID='A3NMH1KTLG7CWX', rating=5.0, timestamp=1398816000),
 Row(bookID=None, userID='A2LI5026JCXQBA', rating=4.0, timestamp=1398729600),
 Row(bookID=None, userID='AHNMXYVRDN1R9', rating=5.0, timestamp=1394323200),
 Row(bookID=None, userID='A2CAVTNQA2Y3IJ', rating=5.0, timestamp=1384560000),
 Row(bookID=None, userID='A2685NTFXLJJ1T', rating=5.0, timestamp=1377475200),
 Row(bookID=None, userID='A17TBLPM7H401J', rating=4.0, timestamp=1374364800),
 Row(bookID=None, userID='A1840OJGNFSBSN', rating=5.0, timestamp=900460800),
 Row(bookID=None, userID='A3ONKN7GMHG6K2', rating=5.0, timestamp=1400630400),
 Row(bookID=None, userID='A4LSI6PTX23BE', rating=5.0, timestamp=1400284800)]

The number of rows with missing values represent about 13% of the entire dataset.   
Since there isn't a straightforward way of handling the missing values without affecting the results of the recommendations, the rows will simply be dropped.

In [9]:
# Drop missing values
books = books.dropna()

# Verify count
books.count()

44639628

After dropping rows with missing values, we still have around ~44.6M rows of data left to work with.

<hr>

### Subset Dataset based on Books Titles
Due to the large size of the dataset and processing issues, the dataset will be downsampled to about 1,000,000 rows based on books.

**Rationale**:
Since each book on average received about 17 distinct ratings, downsampling the dataset to about 1M rows will require about 58,000 unique books.  
This is still a very large dataset and should be able to return decent results for our recommendation engine.

Convert DF to RDD for Mapping UserIDs

In [10]:
# Convert DF to RDD 
books_rdd = books.rdd.map(list)
books_rdd.first()

[1713353, 'A1C6M8LCIX4M6M', 5.0, 1123804800]

In [12]:
# Get all book IDs
book_ids = books_rdd.map(lambda x: x[0])

# Verify result
book_ids.first()



1713353

Get all unique bookIDs for mapping.

In [13]:
# Get all unique book IDs
book_ids = book_ids.distinct()

# Count total book titles 
total_books = book_ids.count()

print(f"There are {total_books} total books in the dataset.")

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=22478Kb max_used=22551Kb free=108593Kb
 bounds [0x00000001071e0000, 0x0000000108820000, 0x000000010f1e0000]
 total_blobs=9475 nmethods=8526 adapters=861
 compilation: disabled (not enough contiguous free space left)


There are 2266596 total books in the dataset.


The number of unique users in the dataset is 14M+. Now each unique alphanumeric userID can be mapped to a new unique integer ID.

In [26]:
# Get a Random Sample of 58,000 books from list of unique books IDs
book_ids_58k = book_ids.sample(withReplacement=False, fraction=0.026, seed=42)

In [27]:
book_ids_58k.count()

58851

In [32]:
# Broadcast values to nodes and perform collect transformation to be used in filter
broadcasted_book_ids = sc.broadcast(set(book_ids_58k.collect()))

In [33]:
# Get user ids and ratings for subset of 58K books
books_58k = books_rdd.filter(lambda x: x[0] in broadcasted_book_ids.value)

In [35]:
books_58k.count()

1182213

After successfully filtering, the dataset has been reduced to just over 1M rows, and was downsampled based on the book IDs as opposed to the user IDs. 

In [15]:
# Transform to Mllib Rating object
ratings = books_rdd.map(lambda r: Row(userID=int(broadcasted_dict.value.get(r[1])), 
                                      bookID=int(r[0]), 
                                      rating=float(r[2]), 
                                      timestamp=r[3]))

In [16]:
# Verify correctly formatted Row object
ratings.first()

Row(userID=7, bookID=1713353, rating=5.0, timestamp=1123804800)

The data has now been formatted appropriately for the collaborative filtering algorithm:  
Rows of `Rating(int(user), int(product), float(rating), long(timestamp))`

Convert Back to DF

In [17]:
schema = StructType([
    StructField('userID', IntegerType(), False),
    StructField('bookID', IntegerType(), False),
    StructField('rating', FloatType(), False),
    StructField('timestamp', LongType(), False),
])

In [18]:
# Convert back to DF for more efficient processing
ratings_df = spark.createDataFrame(ratings, schema=schema)

In [19]:
ratings_df.show(10)

+------+-------+------+----------+
|userID| bookID|rating| timestamp|
+------+-------+------+----------+
|     7|1713353|   5.0|1123804800|
|    15|1713353|   5.0|1112140800|
|    12|1713353|   5.0|1081036800|
|     3|1713353|   5.0|1077321600|
|    31|1713353|   5.0|1475452800|
|     5|1713353|   5.0|1469750400|
|     8|1713353|   5.0|1466380800|
|     0|1713353|   5.0|1461456000|
|     6|1713353|   5.0|1455408000|
|    14|1713353|   5.0|1453593600|
+------+-------+------+----------+
only showing top 10 rows



In [20]:
# Confirm dtypes
ratings_df.dtypes

[('userID', 'int'),
 ('bookID', 'int'),
 ('rating', 'float'),
 ('timestamp', 'bigint')]

In [21]:
type(ratings_df)

pyspark.sql.dataframe.DataFrame

Subset Data for Testing with Various Sizes

In [22]:
# Get Fractions for 20M, 10M, 1M and 500K rows
frac_20M, frac_10M, frac_1M, frac_500K, frac_10K = [0.4477, 0.2238, 0.0224, 0.0112, 0.0224]


# Subset data 
books_subset_10K = ratings_df.sample(withReplacement=False, fraction=frac_10K)
books_subset_500K = ratings_df.sample(withReplacement=False, fraction=frac_500K)
books_subset_1M = ratings_df.sample(withReplacement=False, fraction=frac_1M)
books_subset_10M = ratings_df.sample(withReplacement=False, fraction=frac_10M)
books_subset_20M = ratings_df.sample(withReplacement=False, fraction=frac_20M)

### Split data into Train/Test Sets


In [23]:
train, test = books_subset_10K.randomSplit([0.7, 0.3], seed=42)

## Build Recommendation Model using ALS

These params are chosen as a baseline based on benchmarking results [here.](http://mymedialite.net/examples/datasets.html)


In [24]:
# Define model hyperparametes
RANK = 10
MAX_ITER = 15
REG_PARAM = 0.05

# Define columns
COL_USER = "userID"
COL_ITEM = "bookID"
COL_RATING = "rating"
COL_PREDICTION = "prediction"
COL_TIMESTAMP = "timestamp"

In [33]:
# num of recommended books
K = 5

In [34]:
# # Build a baseline model with 10K rows only
# als = ALS(
#     maxIter=MAX_ITER,
#     rank=RANK,
#     regParam=REG_PARAM,
#     userCol=COL_USER,
#     itemCol=COL_ITEM,
#     ratingCol=COL_RATING,
#     coldStartStrategy="drop"
# )

# model = als.fit(train)

23/11/19 11:02:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/19 11:02:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [49]:
# save model
# model.save('../models/baseline')

### Model Evaluation

In [ ]:
# Load model 
# loaded_model = ALS.load('../models/model1')

In [36]:
# Predict with model
# predictions = model.transform(test)

In [37]:
# # Evaluate the model
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

In [38]:
# print(f"The root mean squared error for the model is: {rmse}")

The root mean squared error for the model is: 4.719012752097297


In [40]:
# Evaluate the model on test data
# predicted_ratings = model.predictAll(test).map(lambda x: ((x[0], x[1]), x[2]))

In [ ]:

# print(
#     "RMSE score = {}".format(evaluations.rmse()),
#     "MAE score = {}".format(evaluations.mae()),
#     "R2 score = {}".format(evaluations.rsquared()),
#     "Explained variance score = {}".format(evaluations.exp_var()),
#     sep="\n"
# )

Tuning Model Hyperparameters

In [25]:
# Build generic ALS Model
als2 = ALS(userCol=COL_USER, 
          itemCol=COL_ITEM, 
          ratingCol=COL_RATING,
          coldStartStrategy="drop", 
          nonnegative=True, 
          implicitPrefs=False)

In [26]:
# Define hyperparams for CV
param_grid = ParamGridBuilder()\
             .addGrid(als2.rank, [5, 20, 40, 80])\
             .addGrid(als2.maxIter, [5, 100, 250, 300])\
             .addGrid(als2.regParam, [0.05, 0.1, 1.5])\
             .build()

In [27]:
# Evaluate Model Performance
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating", 
                                predictionCol="prediction")

In [28]:
# Perform Cross-Validation for best hyperparams
cv = CrossValidator(estimator=als2,
                    estimatorParamMaps=param_grid,
                    evaluator=evaluator,
                    numFolds=5, 
                    parallelism=4)

In [29]:
# Run the cv on training data
model = cv.fit(train)

23/11/19 17:56:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/19 18:57:59 ERROR Instrumentation: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.io.ObjectStreamClass$FieldReflector.getPrimFieldValues(ObjectStreamClass.java:2171)
	at java.io.ObjectStreamClass.getPrimFieldValues(ObjectStreamClass.java:1404)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1533)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOut

Py4JJavaError: An error occurred while calling o408.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.io.ObjectStreamClass$FieldReflector.getPrimFieldValues(ObjectStreamClass.java:2171)
	at java.io.ObjectStreamClass.getPrimFieldValues(ObjectStreamClass.java:1404)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1533)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:

23/11/19 18:59:13 ERROR Executor: Exception in task 0.0 in stage 648.0 (TID 4233)
java.lang.StackOverflowError
	at java.io.ObjectInputStream$BlockDataInputStream.readInt(ObjectInputStream.java:3358)
	at java.io.ObjectInputStream.readHandle(ObjectInputStream.java:1806)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1867)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2209)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1692)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2454)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2378)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2236)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1692)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2454)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2378)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInp

In [ ]:
# Extract best combinations of values from CV
# best_model = model.bestModel

## Surprise Library

In [31]:
books_1M = books_subset_1M.toPandas()

ConnectionRefusedError: [Errno 61] Connection refused